## 7.5 NotyApp

Aplikacja będzie zawierać prostą listę notatek/zadań zapisaną w bazie danych `SQLite`. Lista będzie dostępna z poziomu `Widget` jak i w samej aplikacji. Przyjrzymy się jak zaimplemntować podstawowe elementy komunikacji między widgetem a naszą aplikacją, oraz jak aktualizować dane wyświetlane w `ListView` na `Widget`.

### **Layout**

Rozpocznijmy od utworzenia layoutów dla samego widgetu oraz dla pojedynczego elementu `ListView` (podobnie jak w `RecyclerView`). Layout widgetu będzie składał się z samego `ListView` oraz pola `TextView`, które będzie wyświetlane gdy lista jest pusta

In [ ]:
// noty_widget_provider.xml
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    style="@style/Widget.NotyJava.AppWidget.Container"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:theme="@style/Theme.NotyJava.AppWidgetContainer">

    <ListView
        android:id="@+id/listViewWidget"
        android:layout_width="match_parent"
        android:layout_height="match_parent"/>

    <TextView
        android:id="@+id/emptyViewTextView"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:textSize="20sp"
        android:text="lista jest pusta"/>
</RelativeLayout>

Następnie dodajmy layout pojedynczego elementu listy - wstępnie będzie to tylko pole `TextView`

In [ ]:
// item_view.xml
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="wrap_content">

    <TextView
        android:id="@+id/itemListTextView"
        android:layout_width="match_parent"
        android:layout_height="110dp"
        android:gravity="center"
        android:text="przykład długiego tekstu"
        android:background="@color/teal_200"
        android:textColor="@color/black"/>

</RelativeLayout>

Dodajmy opis widgetu do katalogu `xml`

In [ ]:
// noty_widget_provider_info.xml
<appwidget-provider xmlns:android="http://schemas.android.com/apk/res/android"
    android:initialLayout="@layout/noty_widget_provider"
    android:minHeight="110dp"
    android:minResizeHeight="40dp"
    android:minWidth="110dp"
    android:resizeMode="vertical|horizontal"
    android:updatePeriodMillis="36000000"
    android:widgetCategory="home_screen" />

Wstępnie dane będziemy pobierać (jak zwykle) z `DataProvider`

In [ ]:
public final class DataProvider {
    private DataProvider(){}

    public static final String[] data = {
        "notatka 1", 
        "notatka 2", 
        "notatka 3", 
        "notatka 4", 
        "notatka 5", 
        "notatka 6", 
        "notatka 7", 
        "notatka 8", 
        "notatka 9"};

    public static ArrayList<String> dummyData = new ArrayList<>(Arrays.asList(data));
}

### **RemoteViewsService**

Rozpoczniemy od implementacji odpowiedniej usługi - jest ona niezbędna ponieważ widget działa na kompletnie innym procesie i nie możemy się komunikować wprost. Klasa `RemoteViewsService` jest usługą, z którą widgety będą się łączyć aby adapter mógł otrzymać instancje `RemoteViews`

In [ ]:
public class NotyWidgetService extends RemoteViewsService {
    @Override
    public RemoteViewsFactory onGetViewFactory(Intent intent) {
        return null;
    }
}

Musimy zaimplementować jedną metodę `onGetViewFactory`, która zwraca obiekt dostarczający dane. W tym celu zaimplementujemy klasę implementującą interfejs `RemoteViewsFactory` zawierającą szerego niezbędnych metod
- `onCreate` - tutaj łączymy się z bazą danych, wywoływana zaraz po konstruktorze
- `onDataSetChanged` - wywoływana gdy adapter wywołuje `notifyDataSetChanged`
- `onDestroy` - wywoływana w momencie usunięcia powiązania z ostatnim adapterem
- `getCount` - zwraca liczbę elementów w kolekcji
- `getViewAt` - zwraca obiekt `View` powiązany z daną pozycją
- `getLoadingView` - zezwala na załączenie własnego `View` dla ekranu ładowania
- `getViewTypeCount` - zwraca liczbę **różnych** typów `View` wykorzystywanych przez adapter - w tym przykładzie będziemy wykorzystywać jeden typ
- `getItemId` - zwraca identyfikator - tutaj będzie to odpowiednikiem rzędu na liście
- `hasStableIds` - określa czy identyfikatory pozostają niezmienne przy zmianie powiązanych z nim danych

In [ ]:
static class NotyWidgetItemFactory implements RemoteViewsFactory {
    
            @Override
        public void onCreate() {
        }

        @Override
        public void onDataSetChanged() {

        }

        @Override
        public void onDestroy() {
        }

        @Override
        public int getCount() {
        }

        @Override
        public RemoteViews getViewAt(int position) {

        }

        @Override
        public RemoteViews getLoadingView() {
            return null;
        }

        @Override
        public int getViewTypeCount() {
        }

        @Override
        public long getItemId(int position) {
        }

        @Override
        public boolean hasStableIds() {
        }

Rozpocznijmy od dodania kilku pól

In [ ]:
private final Context context;
private final int appWidgetId;
private ArrayList<String> noteList;

Musimy również zaimplementować konstruktor w którym przekażemy `context` oraz `appWidgetId`

In [ ]:
public NotyWidgetItemFactory(Context context, Intent intent){
    this.context = context;
    this.appWidgetId = intent.getIntExtra(AppWidgetManager.EXTRA_APPWIDGET_ID,
            AppWidgetManager.INVALID_APPWIDGET_ID);
}

- W metodzie `onCreate` zainicjujemy listę notatek - w pierwszej fazie będzie to lista z klasy `DataProvider`

In [ ]:
@Override
public void onCreate() {
    noteList = DataProvider.dummyData;
}

- `getCount` zwraca wielkość listy

In [ ]:
@Override
public int getCount() {
    return noteList.size();
}

- W tym przykładzie będziemy wykorzystywać z domyślnego widoku ładowania, więc metoda zwraca `null`

In [ ]:
@Override
public RemoteViews getLoadingView() {
    return null;
}

- Posiadamy jeden typ `View`, więc zwracamy `1`

In [ ]:
@Override
public int getViewTypeCount() {
    return 1;
}

- ponieważ w pierwszej fazie będziemy korzystać tylko z listy, posłużymy się pozycją jako identyfikatorem

In [ ]:
@Override
public long getItemId(int position) {
    return position;
}

- nasza kolekcja w tej chwili posiada stabilne `id`

In [ ]:
@Override
public boolean hasStableIds() {
    return true;
}

- w metodzie `getViewAt` zwracamy odpowiedni obiekt

In [ ]:
@Override
public RemoteViews getViewAt(int position) {
    RemoteViews remoteViews = new RemoteViews(context.getPackageName(), R.layout.item_list);
    remoteViews.setTextViewText(R.id.itemListTextView, noteList.get(position));
    return remoteViews;
}

Powracamy do klasy `NotyWidgetService` i w metodzie `onGetViewFactory` zwracamy instancję `NotyWidgetItemFactory`

In [ ]:
public class NotyWidgetService extends RemoteViewsService {

    @Override
    public RemoteViewsFactory onGetViewFactory(Intent intent) {
        return new NotyWidgetItemFactory(getApplicationContext(), intent);
    }
    ...
}

### **AppWidgetProvider**

Drugim niezbędnym elementem będzie `AppWidgetProvider`

In [ ]:
public class NotyWidgetProvider extends AppWidgetProvider {
    @Override
    public void onUpdate(Context context, AppWidgetManager appWidgetManager, int[] appWidgetIds) {
        super.onUpdate(context, appWidgetManager, appWidgetIds);
    }
}

W metodzie `onUpdate` przechodzimy przez wszystkie instancje naszego widgetu

In [ ]:
for (int appWidgetId : appWidgetIds) {

Tworzymy nowy `Intent` przekierowujący do usługi

In [ ]:
    Intent serviceIntent = new Intent(context, NotyWidgetService.class);

następnie dodajmy `id` widgetu oraz wykorzystujemy metodę `setData` - metoda ta wskazuje lokalizację obiektu (przykładowo może być to plik)

In [ ]:
    serviceIntent.putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId);
    serviceIntent.setData(Uri.parse(serviceIntent.toUri(Intent.URI_INTENT_SCHEME)));

Tworzymy nasze `RemoteViews`

In [ ]:
    RemoteViews views = new RemoteViews(context.getPackageName(), R.layout.noty_widget_provider);

ustawiamy adapter i widok dla pustej kolekcji

In [ ]:
    views.setRemoteAdapter(R.id.listViewWidget, serviceIntent);
    views.setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView);

na koniec wywołujemy metodę `updateAppWidget`

In [ ]:
    appWidgetManager.updateAppWidget(appWidgetId, views);

Po wyjściu z pętli `for` wywołujemy metodę superklasy

In [ ]:
    }
    super.onUpdate(context, appWidgetManager, appWidgetIds);

Pełny kod klasy `NotyWidgetProvider`

In [ ]:
public class NotyWidgetProvider extends AppWidgetProvider {
    @Override
    public void onUpdate(Context context, AppWidgetManager appWidgetManager, int[] appWidgetIds) {
        for (int appWidgetId : appWidgetIds) {

            Intent serviceIntent = new Intent(context, NotyWidgetService.class);
            serviceIntent.putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId);
            serviceIntent.setData(Uri.parse(serviceIntent.toUri(Intent.URI_INTENT_SCHEME)));

            RemoteViews views = new RemoteViews(context.getPackageName(), R.layout.noty_widget_provider);
            views.setRemoteAdapter(R.id.listViewWidget, serviceIntent);
            views.setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView);
            appWidgetManager.updateAppWidget(appWidgetId, views);
        }
        super.onUpdate(context, appWidgetManager, appWidgetIds);
    }
}

Do `AndroidManifest` musimy wprowadzić informacje o naszym providerze oraz service.

In [ ]:
<receiver
    android:name=".provider.NotyWidgetProvider"
    android:exported="false">
    <intent-filter>
        <action android:name="android.appwidget.action.APPWIDGET_UPDATE" />
    </intent-filter>

    <meta-data
        android:name="android.appwidget.provider"
        android:resource="@xml/noty_widget_provider_info" />
</receiver>

<service
    android:name=".service.NotyWidgetService"
    android:permission="android.permission.BIND_REMOTEVIEWS" />

Możemy przetestować aplikację i widget.

<table><tr><td><img src="https://media2.giphy.com/media/U0lFKx7L0xtZHrpaLQ/giphy.gif?cid=790b76119fc11ce6d16c4ba7009680f1447b35e164c8012a&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media2.giphy.com/media/1BNcJ99feU4EosbU19/giphy.gif?cid=790b76117e6f80ef480e50e651e2e981d067cfeb27c6e32c&rid=giphy.gif&ct=g" width="150" /></td</tr></table>

### **Odświeżanie**

Poza automatycznym odświeżaniem co 30 minut dodamy odświeżanie na przycisk umieszczony na widgecie. Zmodyfikujmy layout widgetu

In [ ]:
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    style="@style/Widget.NotyJava.AppWidget.Container"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:theme="@style/Theme.NotyJava.AppWidgetContainer">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:orientation="vertical">

        <Button
            android:id="@+id/refreshButtonWidget"
            android:layout_width="match_parent"
            android:backgroundTint="@color/cardview_dark_background"
            android:text="refresh"
            android:layout_height="wrap_content"/>


    <ListView
        android:id="@+id/listViewWidget"
        android:layout_width="match_parent"
        android:layout_height="match_parent" />

    </LinearLayout>

    <TextView
        android:id="@+id/emptyViewTextView"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:textSize="20sp"
        android:gravity="center"
        android:text="lista jest pusta"/>
</RelativeLayout>

Przy odświeżeniu chcemy wywołać (niejawnie) metodę `onDataSetChange` klasy `NotyWidgetService`, więc przedźmy do tej metody i ją nieco zmodyfikujmy

In [ ]:
@Override
public void onDataSetChanged() {
    DataProvider.dummyData.add(
        "Nowa notatka " + (DataProvider.dummyData.size() + 1));
}

Czyli przy każdym wywołaniu tej metody będziemy dodawać nową notatkę do listy - tą funkcjonalność póżniej zmienimy. Wróćmy do klasy `NotyWidgetProvider` i obsłużmy przycisk. W pierwszej kolejności musimy utworzyć `Intent`, określimy w nim akcję którą chcemy wykonać - tutaj będzie to `ACTION_APPWIDGET_UPDATE`.

In [ ]:
Intent intentUpdate = new Intent(context, NotyWidgetProvider.class);
intentUpdate.setAction(AppWidgetManager.ACTION_APPWIDGET_UPDATE);

Następnie musimy przekazać **wszystkie** identyfikatory widgetów

In [ ]:
int[] idArray = new int[]{appWidgetId};
intentUpdate.putExtra(AppWidgetManager.EXTRA_APPWIDGET_IDS, idArray);

Tworzymy `PendingIntent` który wykona transmisję

In [ ]:
PendingIntent pendingUpdate = PendingIntent.getBroadcast(
        context, appWidgetId, intentUpdate,
        PendingIntent.FLAG_UPDATE_CURRENT | PendingIntent.FLAG_MUTABLE);

ustawiamy `setOnClickPendingIntent` na `RemoteViews`

In [ ]:
views.setOnClickPendingIntent(R.id.refreshButtonWidget, pendingUpdate);

na koniec metody `onUpdate` wykonujemy `notifyAppWidgetViewDataChanged`

In [ ]:
appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget);

Pełny kod metody `onUpdate`

In [ ]:
@RequiresApi(api = Build.VERSION_CODES.S)
@Override
public void onUpdate(
    Context context, 
    AppWidgetManager 
    appWidgetManager, 
    int[] appWidgetIds) {
    for (int appWidgetId : appWidgetIds) {

        Intent serviceIntent = new Intent(context, NotyWidgetService.class);
        serviceIntent.putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId);
        serviceIntent.setData(Uri.parse(serviceIntent.toUri(Intent.URI_INTENT_SCHEME)));

        Intent intentUpdate = new Intent(context, NotyWidgetProvider.class);
        intentUpdate.setAction(AppWidgetManager.ACTION_APPWIDGET_UPDATE);

        int[] idArray = new int[]{appWidgetId};
        intentUpdate.putExtra(AppWidgetManager.EXTRA_APPWIDGET_IDS, idArray);

        PendingIntent pendingUpdate = PendingIntent.getBroadcast(
                context, appWidgetId, intentUpdate,
                PendingIntent.FLAG_UPDATE_CURRENT | PendingIntent.FLAG_MUTABLE);

        RemoteViews views = new RemoteViews(
            context.getPackageName(), 
            R.layout.noty_widget_provider);
        views.setRemoteAdapter(R.id.listViewWidget, serviceIntent);
        views.setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView);
        views.setOnClickPendingIntent(R.id.refreshButtonWidget, pendingUpdate);

        appWidgetManager.updateAppWidget(appWidgetId, views);
        appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget);
    }
    super.onUpdate(context, appWidgetManager, appWidgetIds);
}

Możemy przetestować aplikację

<img src="https://media1.giphy.com/media/1MmSrqHNPMJ7t6sJd5/giphy.gif?cid=790b761108493c629628d2f83d3b785b8ada6d0dad9b1f6b&rid=giphy.gif&ct=g" width="150" />

### **ListView onClick**

Kolejnym krokiem będzie modyfikacja elementu listy po kliknięciu. Ponieważ tworzenie `PendingIntent` dla każdego elementu listy jest operacją niezwykle kosztochłonną wykonujemy to w dwóch krokach. W klasie `NotyWidgetProvider` utworzymy `PendingIntentTemplate` - szablon intentu. Oraz `FillIntent` przez który prześlemy potrzebne dane. W pierwszym kroku zdefiniujmy `Intent` w klasie `NotyWidgetProvider` w którym zdefiniujemy akcję.

In [ ]:
Intent clickIntent = new Intent(context, NotyWidgetProvider.class);

Musimy określić identyfikator, który posłuży nam do wykonania akcji

In [ ]:
public static final String ACTION_DONE = "actionDone";

In [ ]:
clickIntent.setAction(ACTION_DONE);

Następnie tworzymy `PendingIntent`

In [ ]:
PendingIntent clickPendingIntent = PendingIntent.getBroadcast(
        context, 0, clickIntent,
        PendingIntent.FLAG_MUTABLE);

oraz ustawiamy `PendingIntentTemplate`

In [ ]:
views.setPendingIntentTemplate(R.id.listViewWidget, clickPendingIntent);

Zmodyfikowana metoda `onUpdate`

In [ ]:
@RequiresApi(api = Build.VERSION_CODES.S)
@Override
public void onUpdate(
    Context context, 
    AppWidgetManager appWidgetManager, 
    int[] appWidgetIds) {
    for (int appWidgetId : appWidgetIds) {

        Intent serviceIntent = new Intent(context, NotyWidgetService.class);
        serviceIntent.putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId);
        serviceIntent.setData(Uri.parse(serviceIntent.toUri(Intent.URI_INTENT_SCHEME)));

        Intent clickIntent = new Intent(context, NotyWidgetProvider.class);
        clickIntent.setAction(ACTION_DONE);
        PendingIntent clickPendingIntent = PendingIntent.getBroadcast(
                context, 0, clickIntent,
                PendingIntent.FLAG_MUTABLE);

        Intent intentUpdate = new Intent(context, NotyWidgetProvider.class);
        intentUpdate.setAction(AppWidgetManager.ACTION_APPWIDGET_UPDATE);

        int[] idArray = new int[]{appWidgetId};
        intentUpdate.putExtra(AppWidgetManager.EXTRA_APPWIDGET_IDS, idArray);

        PendingIntent pendingUpdate = PendingIntent.getBroadcast(
                context, appWidgetId, intentUpdate,
                PendingIntent.FLAG_UPDATE_CURRENT | PendingIntent.FLAG_MUTABLE);

        RemoteViews views = new RemoteViews(
            context.getPackageName(), 
            R.layout.noty_widget_provider);
        views.setRemoteAdapter(R.id.listViewWidget, serviceIntent);
        views.setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView);
        views.setPendingIntentTemplate(R.id.listViewWidget, clickPendingIntent);
        views.setOnClickPendingIntent(R.id.refreshButtonWidget, pendingUpdate);

        appWidgetManager.updateAppWidget(appWidgetId, views);
        appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget);
    }
    super.onUpdate(context, appWidgetManager, appWidgetIds);
}

Przechodzimy do klasy `NotyWidgetService`, w metodzie `getViewAt` tworzymy `FillIntent` i przesyłamy niezbędne dane

In [ ]:
Intent fillIntent = new Intent();
fillIntent.putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId);
fillIntent.putExtra("position", position);
remoteViews.setOnClickFillInIntent(R.id.itemListTextView, fillIntent);

Ostatnim krokiem jest odebranie danych i wykonanie akcji, robimy to w klasie `NotyWidgetProvider` nadpisując metodę `onReceive`

In [ ]:
@Override
public void onReceive(Context context, Intent intent) {
    super.onReceive(context, intent);
}

Wpierw określmy akcję którą będziemy obsługiwać

In [ ]:
if (ACTION_DONE.equals(intent.getAction())){

rozpakujmy dane

In [ ]:
int appWidgetId = intent.getIntExtra(AppWidgetManager.EXTRA_APPWIDGET_ID,
        AppWidgetManager.INVALID_APPWIDGET_ID);
int position = intent.getIntExtra("position", 100);

Pobierzmy instancję `AppWidgetManager`

In [ ]:
AppWidgetManager appWidgetManager = AppWidgetManager.getInstance(context);

i zmodyfikujmy element na liście

In [ ]:
DataProvider.dummyData.set(position, "zmiana");

następnie wykonujemy `notifyAppWidgetViewDataChanged`

In [ ]:
appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget);

Pełna metoda `onReceive`

In [ ]:
@Override
public void onReceive(Context context, Intent intent) {
    if (ACTION_DONE.equals(intent.getAction())){
        int appWidgetId = intent.getIntExtra(AppWidgetManager.EXTRA_APPWIDGET_ID,
                AppWidgetManager.INVALID_APPWIDGET_ID);
        int position = intent.getIntExtra("position", 100);
        AppWidgetManager appWidgetManager = AppWidgetManager.getInstance(context);

        DataProvider.dummyData.set(position, "zmiana");

        appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget);
    }
    super.onReceive(context, intent);
}

Możemy przetestować aplikację - po kliknięciu każdego elementu listy, zostanie on zmieniony ale również zostanie dodany nowy element, ponieważ będzie wywołana metoda `onDataSetChanged`

<img src="https://media2.giphy.com/media/aCLr1hpAQuVMUFKCZb/giphy.gif?cid=790b7611a7a7f667dcf133dcf9cd4bf638e9c5bf8dbc1f31&rid=giphy.gif&ct=g" width="150" />

### **Baza danych**

Notatki będziemy przechowywać w bazie danych, notatkę przeczytaną/zrealizowaną oznaczymy innym kolorem tekstu. Oprócz tego dodamy również godzinę wykonania zadania. Rozpocznijmy od zdefiniowania modelu

In [ ]:
public class NoteModel {
    private int id = 0;
    private String textNote;
    private final LocalTime time;
    private int color = Color.BLACK;

    public NoteModel(String textNote, LocalTime date) {
        this.textNote = textNote;
        this.time = date;
    }

    public NoteModel(int id, String textNote, LocalTime time, int color) {
        this(textNote, time);
        this.id = id;
        this.color = color;
    }

    public String getTextNote() {
        return textNote;
    }

    public LocalTime getTime() {
        return time;
    }

    public int getId() {
        return id;
    }

    public void setTextNote(String textNote) {
        this.textNote = textNote;
    }

    public int getColor() {
        return color;
    }

    public void setColor(int color) {
        this.color = color;
    }
}

Zdefiniujmy również dane testowe w klasie `DataProvider`

In [ ]:
private static final NoteModel[] notes = {
        new NoteModel("notatka 1", LocalTime.of(12, 0)),
        new NoteModel("notatka 2", LocalTime.of(13, 0)),
        new NoteModel("notatka 3", LocalTime.of(21, 0)),
        new NoteModel("notatka 4", LocalTime.of(9, 9)),
        new NoteModel("notatka 5", LocalTime.of(22, 34)),
        new NoteModel("notatka 6", LocalTime.of(11, 22)),
        new NoteModel("notatka 7", localTime)
};

public static ArrayList<NoteModel> dummyData2 = new ArrayList<>(Arrays.asList(notes));

Stwórzmy naszą bazę danych

In [ ]:
public class DBHandler extends SQLiteOpenHelper {

    private static int DATABASE_VERSION = 1;
    private static final String DATABASE_NAME = "notesBDk.db";
    private static final String NOTES_TABLE = "NotesTable";

    private static final String COLUMN_ID = "_id";
    private static final String COLUMN_TEXT = "text";
    private static final String COLUMN_TIME = "time";
    private static final String COLUMN_COLOR = "color";

    public DBHandler(@Nullable Context context) {
        super(context, DATABASE_NAME, null, DATABASE_VERSION);
    }

    @Override
    public void onCreate(SQLiteDatabase db) {
        String CREATE_STUDENTS_TABLE = "CREATE TABLE " +
                NOTES_TABLE +
                "(" +
                COLUMN_ID + " " +
                "INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL," +
                COLUMN_TEXT +
                " TEXT," +
                COLUMN_TIME +
                " TEXT," +
                COLUMN_COLOR +
                " INTEGER" +
                ")";

        db.execSQL(CREATE_STUDENTS_TABLE);
    }

    @Override
    public void onUpgrade(SQLiteDatabase db, int oldVersion, int newVersion) {
        db.execSQL("DROP TABLE IF EXISTS " + NOTES_TABLE);
        onCreate(db);
    }
}

Będziemy potrzebować kilka metod, zacznijmy od dodania nowego wpisu do bazy

In [ ]:
    public void addNote(NoteModel note){
        SQLiteDatabase db = this.getWritableDatabase();

        ContentValues values = new ContentValues();
        values.put(COLUMN_TEXT, note.getTextNote());
        values.put(COLUMN_TIME, note.getTime().toString());
        values.put(COLUMN_COLOR, note.getColor());

        db.insert(NOTES_TABLE, null, values);
        db.close();
    }

Przejdźmy do metody zwracającą listę wszystkich notatek

In [ ]:
    public ArrayList<NoteModel>  getNotes() {
        ArrayList<NoteModel> notes = new ArrayList<>();

        SQLiteDatabase db = this.getReadableDatabase();

        Cursor cursor = db.rawQuery("SELECT * FROM " + NOTES_TABLE, null);

        if (cursor.moveToFirst()) {
            do {
                notes.add(new NoteModel(
                        cursor.getInt(0),
                        cursor.getString(1),
                        LocalTime.parse(cursor.getString(2)),
                        cursor.getInt(3)));
            } while (cursor.moveToNext());
        }

        db.close();
        cursor.close();
        return notes;
    }

Będziemy również aktualizować kolor danej notatki

In [ ]:
    public void updateNote (int id){
        SQLiteDatabase db = this.getWritableDatabase();
        ContentValues contentValues = new ContentValues();
        contentValues.put(COLUMN_COLOR, Color.CYAN);

        db.update(NOTES_TABLE,
                contentValues,
                COLUMN_ID + "=" + id,
                null);

        db.close();
    }

Zainicjujmy naszą bazę za pomocą danych testowych w klasie `MainActivity`

In [ ]:
    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);
        DBHandler dbHandler = new DBHandler(this);
        DataProvider.dummyData2.forEach(dbHandler::addNote);
        dbHandler.close();
    }

Dokonajmy zmian w klasie `NotyWidgetItemFactory`, będziemy potrzebować zmienną `DBHandler`

In [ ]:
private DBHandler dbHandler;

Bazę zainicjujemy w metodzie `onCreate`, tutaj również zainicjujemy lokalną listę

In [ ]:
    @Override
    public void onCreate() {
        dbHandler = new DBHandler(context);
        noteList = dbHandler.getNotes();
    }

Przy każdej zmianie chcemy odświeżyć listę i pobrać aktualne dane

In [ ]:
    @Override
    public void onDataSetChanged() {
        noteList = dbHandler.getNotes();
    }

W metodzie `onDestroy` zamykamy dostęp do bazy

In [ ]:
    @Override
    public void onDestroy() {
        dbHandler.close();
    }

W metodzie `getViewAt` ustawmy tekst oraz kolor tekstu elementów listy

In [ ]:
    RemoteViews remoteViews = 
        new RemoteViews(context.getPackageName(), R.layout.item_list);
    remoteViews.setTextViewText(
        R.id.itemListTextView, 
        noteList.get(position).getTime().toString()
        + "\n" + noteList.get(position).getTextNote());
    remoteViews.setTextColor(
        R.id.itemListTextView,  
        noteList.get(position).getColor());

Przez `FillIntent` będziemy przesyłać `appWidgetId` oraz `id` elementu listy

In [ ]:
        Intent fillIntent = new Intent();
        fillIntent.putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId);
        fillIntent.putExtra("id", noteList.get(position).getId());
        remoteViews.setOnClickFillInIntent(R.id.itemListTextView, fillIntent);
        return remoteViews;

Zmienimy również `getItemId` - teraz będziemy posługiwać się `id` z bazy danych

In [ ]:
        @Override
        public long getItemId(int position) {
            return noteList.get(position).getId(); // identyfikacja
        }

Przejdźmy do klasy `NotyWidgetProvider` i w metodzie `onReceive` zaktualizujmy element listy

In [ ]:
    @Override
    public void onReceive(Context context, Intent intent) {
        if (ACTION_DONE.equals(intent.getAction())){
            int appWidgetId = intent.getIntExtra(AppWidgetManager.EXTRA_APPWIDGET_ID,
                    AppWidgetManager.INVALID_APPWIDGET_ID);
            int id = intent.getIntExtra("id", 100);
            AppWidgetManager appWidgetManager = AppWidgetManager.getInstance(context);

            DBHandler dbHandler = new DBHandler(context);
            dbHandler.updateNote(id);

            appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget);
        }
        super.onReceive(context, intent);
    }

Możemy przetestować aplikację

<img src="https://media2.giphy.com/media/jCI4kz73h3KJEs0AJj/giphy.gif" width="150" />